# Time-varying distributions with mean shift

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


In [1]:

# **H1** scenario
mu_x, sigma_x = 0.3, 0.01
mu_shift = 0.35
y2=np.empty(500)
y1 = np.random.normal(mu_x, sigma_x, 500)
y2[:300] = np.random.normal(mu_x, sigma_x, 300)
y2[300:] = np.random.normal(mu_shift, sigma_x, 200)

# **H0** scenario
z1 = y1 
z2 = np.random.normal(mu_x, sigma_x, 500)


In [ ]:
import bet_ons_e3
from bet_ons_e3 import call_process_ons
import bet_ftrl_barrier_e3
from bet_ftrl_barrier_e3 import call_process_ftrl
import bet_optimistic_ftrl_barrier_e3
from bet_optimistic_ftrl_barrier_e3 import call_process_op_ftrl

In [3]:

data1=y1
data2=y2
data3=z1
data4=z2
alphas=np.linspace(0.005, 0.1, 20)
iters=300
shift_time = None
output_file='Sythetic_Datasets/exp3/results/sim_exp3.json'
call_process_ons(data1, data2, data3, data4, alphas,iters, shift_time, output_file)
call_process_ftrl(data1, data2, data3, data4, alphas,iters, shift_time, output_file)
call_process_op_ftrl(data1, data2, data3, data4, alphas,iters, shift_time, output_file)

# Get results to plot

In [ ]:

import json
import os
import numpy as np


json_files = ['Sythetic_Datasets/exp3/results/sim_exp3.json']
item_names = ['ONS','FTRL+Barrier', 'Optimistic-FTRL+Barrier']
data_collect = {name: {'rejection_time': [], 'power': [], 'fpr': []} for name in item_names}

for file in json_files:
    with open(file, 'r') as f:
        data = json.load(f)
        for i, item in enumerate(data):  
            data_collect[item_names[i]]['rejection_time'].append(item['rejection_time'])
            data_collect[item_names[i]]['power'].append(item['power'])
            data_collect[item_names[i]]['fpr'].append(item['fpr'])

results = []
for name, metrics in data_collect.items():
    avg_rejection_time = np.mean(metrics['rejection_time'], axis=0).tolist()
    avg_power = np.mean(metrics['power'], axis=0).tolist()
    avg_fpr = np.mean(metrics['fpr'], axis=0).tolist()
    results.append({
        'item_name': name,
        'rejection_time': avg_rejection_time,
        'power': avg_power,
        'fpr': avg_fpr
    })

results_json = json.dumps(results, indent=4)

with open('Sythetic_Datasets/exp3/results_to_plot/sim_exp3.json', 'w') as f:
    f.write(results_json)


# Plot Results

In [ ]:


with open('Sythetic_Datasets/exp3/results_to_plot/sim_exp3.json', 'r') as file:
    items = json.load(file)

df_list = []
for item in items:
    df = pd.DataFrame({
        'rejection_time': item['rejection_time'],
        'fpr': item['fpr'],
        'name': item['item_name'], 
        'alpha': np.linspace(0.005, 0.1, len(item['fpr']))  
    })
    df_list.append(df)

fig, ax = plt.subplots(1, 2, figsize=(13, 4.2))  
i=0
markers = ["*", "s", "^"]
for df in df_list:
    ax[0].plot(df['rejection_time'], df['fpr'],ls='--', lw=2,  marker=markers[i], label=df['name'].iloc[0], markersize=9)
    ax[1].plot(df['alpha'], df['fpr'], ls='--', lw=2, marker=markers[i], label=df['name'].iloc[0], markersize=9)
    i += 1

ax[0].tick_params(axis='both', labelsize=20, which='major', length=10,  width=2)
ax[0].set_ylim(-0.005,0.105)
ax[0].set_yticks(np.arange(0,0.105,0.02))
ax[0].axvline(x=300, color='grey', linestyle='--')
ax[0].set_xlim(280,520)
ax[0].set_xticks(np.arange(300,520, 50))
ax[0].set_xlabel(r'Rejection Time ($\tau$)', fontsize=20)
ax[0].set_ylabel('False Positive Rate (FPR)', fontsize=20)
x = np.linspace(0, 0.1, 500)
y = x 
ax[1].fill_between(x, 0, y, color='yellow', alpha=0.1, zorder=1)  
ax[1].set_ylim(-0.005,0.105)
ax[1].set_yticks(np.arange(0,0.105,0.02))
ax[1].set_xlim(-0.005,0.105)
ax[1].set_xticks(np.arange(0,0.12,0.02))
ax[1].tick_params(axis='both', labelsize=20, which='major', length=10,  width=2)
ax[1].plot([0, 0.1], [0, 0.1], color='k', ls='--',  lw=3)
ax[1].set_xlabel(r'Significance Level ($\alpha$)', fontsize=20)
ax[1].set_ylabel('False Positive Rate (FPR)', fontsize=20)
handles, labels = next(ax.flat).get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.49, -0.07), fancybox=True, shadow=True,
             ncol=5, fontsize=20, labelspacing=0.1, handletextpad=0.5, handlelength=1)
plt.subplots_adjust(wspace=0.4)  
for axis in ax:  
    for spine in axis.spines.values():
        spine.set_linewidth(2)  
